In [1]:
import dataAck
import portfolio
import time
import hashlib
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
##GET PREDICTIONS OF ALL MODELS IN A PORTFOLIO

allPortfolios = portfolio.getPortfolios()
allPortfolios

[{'benchmark': 'EWZ',
  'description': 'Just trade underlying',
  'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f'},
 {'benchmark': 'VO',
  'description': 'VO 8-13-17',
  'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f'}]

In [3]:
allModels = []
modelsInPortfolio = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    allModels += models
    for model in models:
        print(model.describe())
    

{'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f', 'description': 'Just trade underlying', 'benchmark': 'EWZ'}
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)


In [4]:
tickersRequired = []
tickersTraded = []
for mod in allModels:
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    
pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])

((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
GDX
VO
TIP
SHY
EWZ
XLV
IWF


In [5]:
from google.cloud import datastore, storage, logging
import time
import params
import hashlib
import pandas as pd
def downloadAggregatePredictions(model):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.aggregatePrediction)
            
            query.add_filter('modelHash', '=', hashlib.sha224((str(model.describe())).encode('utf-8')).hexdigest())
            retrievedPredictions = list(query.fetch())
            days = []
            predictions = []
            for pred in retrievedPredictions:
                days.append(pred["predictionDay"])
                predictions.append(pred["aggregatePrediction"])
            
            return pd.DataFrame(predictions, index=days, columns=[str(model.describe())]).sort_index()
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [6]:
aggregateReturns = None
aggregatePredictions = None
for model in allModels:
    preds = downloadAggregatePredictions(model).tz_localize(None)
    dailyFactorReturn = dataAck.getDailyFactorReturn(model.inputSeries.targetTicker, joinedData)
    transformedPreds = preds.join(dailyFactorReturn).dropna()
    returnStream = pd.DataFrame(transformedPreds.apply(lambda x:x[0] * x[1], axis=1), columns=[portfolio.getModelHash(model)])
    preds.columns = [portfolio.getModelHash(model)]
    if aggregateReturns is None:
        aggregateReturns = returnStream
        aggregatePredictions = preds
    else:
        aggregateReturns = aggregateReturns.join(returnStream)
        aggregatePredictions = aggregatePredictions.join(preds)

In [7]:
aggregateReturns

,e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac,bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e,78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc,341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9,0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3,11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e,2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6
2009-03-02,0.013616,0.003404,-0.000000,0.007314,-0.000000,0.007314,-0.000000
2009-03-03,0.035261,0.000000,0.036534,-0.036534,-0.018267,-0.018267,0.000000
2009-03-04,-0.013245,-0.006623,-0.047188,0.047188,0.047188,-0.000000,-0.000000
2009-03-05,-0.000000,-0.001074,-0.000000,0.002089,0.002089,-0.001044,-0.000000
2009-03-06,-0.000000,-0.006638,-0.000000,-0.000000,0.005531,-0.011062,-0.000000
2009-03-09,0.000000,0.078182,0.066825,0.066825,-0.033412,0.066825,0.066825
2009-03-10,-0.000000,-0.002361,0.000000,0.005102,-0.002551,0.005102,0.005102
2009-03-11,0.016058,0.032116,0.000000,0.000000,-0.021433,0.042865,0.042865
2009-03-12,-0.000000,-0.006223,0.000000,-0.005408,-0.002704,0.005408,0.000000
2009-03-13,-0.000000,-0.007910,0.011565,-0.000000,0.005783,-0.005783,-0.000000


In [8]:
aggregatePredictions

,e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac,bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e,78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc,341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9,0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3,11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e,2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6
2009-03-02,1.00,0.25,0.0,-1.0,0.0,-1.0,0.0
2009-03-03,0.50,0.00,1.0,-1.0,-0.5,-0.5,0.0
2009-03-04,0.50,0.25,1.0,-1.0,-1.0,0.0,0.0
2009-03-05,0.00,0.50,0.0,-1.0,-1.0,0.5,0.0
2009-03-06,0.00,0.50,0.0,0.0,-0.5,1.0,0.0
2009-03-09,0.00,1.00,1.0,1.0,-0.5,1.0,1.0
2009-03-10,0.00,1.00,0.0,1.0,-0.5,1.0,1.0
2009-03-11,0.50,1.00,0.0,0.0,-0.5,1.0,1.0
2009-03-12,0.00,1.00,0.0,-1.0,-0.5,1.0,0.0
2009-03-13,0.00,1.00,-1.0,0.0,-0.5,0.5,0.0


In [17]:

def storePortfolioAllocation(portfolioKey, predictionDay, algorithmWeights, tickerAllocation, transformedAlgoPrediction, shouldReturn = False):
    toUpload = {}
    toUpload["portfolio"] = portfolioKey
    toUpload["predictionDay"] = predictionDay
    
    for item in algorithmWeights:
        toUpload["algo_weight_" + item] = algorithmWeights[item]
    
    for item in transformedAlgoPrediction:
        toUpload["algo_" + item] = transformedAlgoPrediction[item]
    
    for item in tickerAllocation:
        toUpload["ticker_" + item] = tickerAllocation[item]
        
    ##SCALE TICKER ALLOCATIOn
    totalAllocation = 1.0#sum([abs(tickerAllocation[item]) for item in tickerAllocation])
    for item in tickerAllocation:
        toUpload["scaled_ticker_" + item] = abs(tickerAllocation[item])/totalAllocation
    
    ##UPLOAD ORGANISM OBJECT
    while True:
        try:
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            predictionHash = hashlib.sha224((str(portfolioKey) + " " + str(toUpload["predictionDay"])).encode('utf-8')).hexdigest()
            key = datastoreClient.key(params.portfolioAllocation, predictionHash) #NEED TO HASH TO ENSURE NON-OVERLAPPING PREDICTIONS
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            if shouldReturn == False:
                datastoreClient.put(organismToStore)
            else:
                return organismToStore
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [18]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = len(aggregateReturns) - windowSize - startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
        historicalWeights = pd.concat([historicalWeights, thisWeights])
        i += 1
    return hrpReturns, historicalWeights

In [19]:
def storeHistoricalAllocations(portfolioKey, modelsInPortfolio, historicalWeights, aggregatePredictions):

    aggregatePredictions = aggregatePredictions.dropna()
    allocationsToStore = []
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        transformedAlgoPrediction = {}
        for model in modelsInPortfolio:
            if model.inputSeries.targetTicker not in netPosition:
                netPosition[model.inputSeries.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.inputSeries.targetTicker] += weights[portfolio.getModelHash(model)] * aggregatePredictions.loc[historicalWeights.index[i]][portfolio.getModelHash(model)]
            transformedAlgoPrediction[portfolio.getModelHash(model)] = weights[portfolio.getModelHash(model)] * aggregatePredictions.loc[historicalWeights.index[i]][portfolio.getModelHash(model)]
        allocationsToStore.append(storePortfolioAllocation(portfolioKey, historicalWeights.index[i], weights.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
    portfolio.storeManyItems(allocationsToStore)

In [20]:
##GENERATE HISTORICAL WEIGHTS -> ONLY NEED TO DO FOR NEW PORTFOLIO
for portfolioKey in modelsInPortfolio:
    print(portfolioKey)
    hrpReturns, historicalWeights = produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 22, startIndex=None, maxWindowSize=True)
    print("COMPUTED HISTORICAL WEIGHTS")
    storeHistoricalAllocations(portfolioKey, modelsInPortfolio[portfolioKey], historicalWeights, aggregatePredictions)

COMPUTED HISTORICAL WEIGHTS
COMPUTED HISTORICAL WEIGHTS


In [17]:
##GENERATE WEIGHTS FOR PORTFOLIO TODAY
allocationsToStore = []
for portfolioKey in modelsInPortfolio:
    hrpReturns, historicalWeights = produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 22, startIndex=3, maxWindowSize=True)
    print(portfolioKey, historicalWeights.iloc[-1])
    todayWeight = historicalWeights.iloc[-1]
    netPosition = {}
    transformedAlgoPrediction = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        transformedAlgoPrediction[portfolio.getModelHash(model)] = todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    allocationsToStore.append(storePortfolioAllocation(portfolioKey, portfolio.getToday(), todayWeight.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
portfolio.storeManyItems(allocationsToStore)

7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f 0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3    0.217790
11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e    0.286753
2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6    0.140269
341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9    0.125790
78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc    0.229397
Name: 2017-08-08 00:00:00, dtype: float64
7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f {'VO': 0.32785386562432417}


In [18]:
portfolio.getPortfolioAllocations("7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f", portfolio.getToday())

[<Entity('walkforward_portfolio_allocation', 'd6a486a76ac7ecf201f4daa8d002973cbfe4088ab240a12c7f5465be') {'algo_weight_2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6': 0.14026903240532929, 'algo_2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6': 0.14026903240532929, 'scaled_ticker_VO': 1.0, 'algo_weight_341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9': 0.12578999106320612, 'algo_341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9': -0.12578999106320612, 'algo_weight_78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc': 0.22939716696915574, 'algo_78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc': 0.0, 'ticker_VO': 0.32785386562432417, 'predictionDay': datetime.datetime(2017, 8, 13, 0, 0, tzinfo=<UTC>), 'portfolio': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'algo_weight_0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3': 0.21779033164214712, 'algo_0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3': 0.21779033164